In [1]:
import sqlalchemy as sql
import pandas as pd
import datetime as dt

In [2]:
engine = sql.create_engine('sqlite:///mta_data.db')
insp = sql.inspect(engine)
insp.get_table_names()

['mta_data']

In [3]:
mta_data = pd.read_sql('SELECT * FROM mta_data;', engine)
mta_data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/21/2021,00:00:00,REGULAR,7622548,2607689
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/21/2021,04:00:00,REGULAR,7622561,2607697
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/21/2021,08:00:00,REGULAR,7622573,2607718
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/21/2021,12:00:00,REGULAR,7622604,2607766
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/21/2021,16:00:00,REGULAR,7622715,2607802


In [4]:
mta_data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [5]:
unique_station_linenames = pd.read_sql('SELECT DISTINCT LINENAME, STATION FROM mta_data GROUP BY STATION', engine)
unique_station_linenames.head()

,LINENAME,STATION
0,L,1 AV
1,BC,103 ST
2,7,103 ST-CORONA
3,JZ,104 ST
4,6,110 ST


In [24]:
station_lines = pd.read_sql('SELECT STATION, COUNT(DISTINCT LINENAME) as total_entrances FROM mta_data GROUP BY STATION ', engine)
station_lines.head()

,STATION,total_entrances
0,1 AV,1
1,103 ST,3
2,103 ST-CORONA,1
3,104 ST,2
4,110 ST,1


In [7]:
station_lines.shape

(379, 2)

In [8]:
station_lines[station_lines['total_entrances']>1].shape

(70, 2)

In [9]:
station_lines[station_lines['total_entrances']>1].head(30)

,STATION,total_entrances
1,103 ST,3
3,104 ST,2
5,111 ST,3
6,116 ST,3
9,125 ST,4
10,135 ST,2
13,14 ST,3
14,14 ST-UNION SQ,2
15,145 ST,3
19,155 ST,2


In [10]:
penn_station = pd.read_sql('SELECT * FROM mta_data WHERE STATION LIKE "%PENN%" GROUP BY LINENAME', engine)
penn_station

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R135,R031,01-00-00,34 ST-PENN STA,123,IRT,08/21/2021,00:00:00,REGULAR,16603992,22271631
1,R138,R293,00-00-00,34 ST-PENN STA,123ACE,IRT,08/21/2021,02:00:00,REGULAR,1159524,354091
2,R632,R067,00-00-00,PENNSYLVANIA AV,3,IRT,08/21/2021,00:00:00,REGULAR,1315767,1312480
3,N067,R012,00-00-00,34 ST-PENN STA,ACE,IND,08/21/2021,00:00:00,REGULAR,279314,823670


In [11]:
penn_station = pd.read_sql('SELECT * FROM mta_data WHERE STATION LIKE "%PENN STA" OR STATION LIKE "PENN STA%" GROUP BY STATION', engine)
penn_station.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,N067,R012,00-00-00,34 ST-PENN STA,ACE,IND,08/21/2021,00:00:00,REGULAR,279314,823670


In [12]:
penn_station_lines = pd.read_sql('SELECT STATION, LINENAME FROM mta_data WHERE STATION="34 ST-PENN STA" GROUP BY LINENAME', engine)
penn_station_lines.head()

,STATION,LINENAME
0,34 ST-PENN STA,123
1,34 ST-PENN STA,123ACE
2,34 ST-PENN STA,ACE


In [13]:
penn_station_turnstiles = pd.read_sql('SELECT DISTINCT SCP, STATION, LINENAME, UNIT FROM mta_data WHERE STATION = "34 ST-PENN STA" GROUP BY UNIT, SCP, STATION', engine)
penn_station_turnstiles.tail()

,SCP,STATION,LINENAME,UNIT
97,01-00-01,34 ST-PENN STA,123ACE,R293
98,01-00-02,34 ST-PENN STA,123ACE,R293
99,01-00-03,34 ST-PENN STA,123ACE,R293
100,01-06-00,34 ST-PENN STA,123ACE,R293
101,01-06-01,34 ST-PENN STA,123ACE,R293


In [14]:
penn_station_turnstile_totals = pd.read_sql('SELECT COUNT(DISTINCT SCP), STATION, LINENAME, UNIT FROM mta_data WHERE STATION = "34 ST-PENN STA" GROUP BY LINENAME', engine)
penn_station_turnstile_totals.head()

,COUNT(DISTINCT SCP),STATION,LINENAME,UNIT
0,21,34 ST-PENN STA,123,R031
1,25,34 ST-PENN STA,123ACE,R293
2,50,34 ST-PENN STA,ACE,R012


In [20]:
penn_station_unique = pd.read_sql("SELECT COUNT(DISTINCT DATE), STATION, LINENAME FROM mta_data WHERE STATION = '34 ST-PENN STA' GROUP BY LINENAME", engine)
penn_station_unique.head()

,COUNT(DISTINCT DATE),STATION,LINENAME
0,91,34 ST-PENN STA,123
1,91,34 ST-PENN STA,123ACE
2,91,34 ST-PENN STA,ACE


In [21]:
st34_8ave = pd.read_sql('SELECT * FROM mta_data WHERE STATION LIKE "%34 ST%" GROUP BY LINENAME', engine)
st34_8ave

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R135,R031,01-00-00,34 ST-PENN STA,123,IRT,08/21/2021,00:00:00,REGULAR,16603992,22271631
1,R138,R293,00-00-00,34 ST-PENN STA,123ACE,IRT,08/21/2021,02:00:00,REGULAR,1159524,354091
2,R550,R072,00-00-00,34 ST-HUDSON YD,7,IRT,08/21/2021,00:00:00,REGULAR,141085,370211
3,N067,R012,00-00-00,34 ST-PENN STA,ACE,IND,08/21/2021,00:00:00,REGULAR,279314,823670
4,A022,R022,01-00-00,34 ST-HERALD SQ,BDFMNQRW,BMT,08/21/2021,00:00:00,REGULAR,28968470,12341896


In [16]:
time_sq_sta = pd.read_sql('SELECT * FROM mta_data WHERE STATION LIKE "%TIMES%" GROUP BY LINENAME', engine)
time_sq_sta

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R147,R033,04-00-00,TIMES SQ-42 ST,1237ACENQRS,IRT,08/21/2021,00:00:00,REGULAR,18907037,15980332
1,R143,R032,02-00-00,TIMES SQ-42 ST,1237ACENQRSW,IRT,08/21/2021,00:00:00,REGULAR,3180834,6027842
2,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,08/21/2021,00:00:00,REGULAR,10076197,12680431
